# Tooltip calibration




See : p17 (Section 4.3) of

https://www.researchgate.net/profile/Mihran_Tuceryan/publication/3410747_Calibration_Requirements_and_Procedures_for_a_Monitor-Based_Augmented_Reality_System



In [3]:
import numpy as np
from math import sqrt


In [12]:

#R = matrix (nx3x3) of n rotation matrices for measured locations
#p = matrix (nx3) of n translation vectors for measured location


def tool_offset(R, P):

    #Form matrices
    rows = R.shape[0]
    I = np.eye(3)
    R = R.reshape(3*rows,3)
    IR = I
    
    for i in range(rows-1):
        IR = np.vstack((IR,I))
    IR = np.hstack((IR,-R))
    IP = P.reshape(3*rows, 1)

    #Solve
    print("\nIR\n")
    print(IR)

    print("\nIP\n")
    print(IP)
    

    IRinv = np.linalg.pinv(IR)

    print("\nIRinv\n")
    print(IRinv)
    
    pwt = np.dot(IRinv, IP)

              
    return pwt[0:3, :].T, pwt[3:6, :].T #Split into two points


    

The next cell does a simple test with random data - calculating random rotation matrices and measurement positions, and a fixed world and tool position for the rotation point

In [13]:

#
# Test with random data
#

pw = np.random.rand(1,3)
pt = np.random.rand(1,3)

print("Target")
print("\nPw (world coordinate)\n")
print(pw)
print("\nPt (tool coordinate)\n")
print(pt)
print("---")

# number of points
n = 6

pm = np.zeros((n,3))   #Measured point 
Rm = np.zeros((n,3,3)) #Measure rotation


for i in range(n):
    # make R a proper rotation matrix, force orthonormal
    R = np.mat(np.random.rand(3,3))
    U, S, Vt = np.linalg.svd(R)
    R = U*Vt
    
    # remove reflection
    if np.linalg.det(R) < 0:
       Vt[2,:] *= -1
       R = U*Vt

    # make the arrays of simulated Rotations and Positions
    Rm[i,:,:]=R
    pm[i,:] =  pw - (R*pt.T).T
    

print("Measurements")
print("\nRotations\n")
print(Rm)
print("\nPositions\n")
print(pm)

print("---")

PW, PT = tool_offset(Rm, pm)

print("Estimated")
print("\nPw (world coordinate)\n")
print(PW)
print("\nPt (tool coordinate)\n")
print(PT)
print("---")



Target

Pw (world coordinate)

[[0.30292798 0.15219941 0.76615217]]

Pt (tool coordinate)

[[0.51655573 0.18973577 0.88424373]]
---
Measurements

Rotations

[[[-0.21063461  0.97709093 -0.03043648]
  [-0.37222016 -0.05137322  0.92672161]
  [ 0.90392766  0.20652871  0.37451392]]

 [[-0.0566121   0.96124598  0.26981705]
  [ 0.39111249 -0.22729882  0.8918331 ]
  [ 0.91860007  0.15601737 -0.36308744]]

 [[ 0.95642918 -0.20764554 -0.20524752]
  [ 0.24892637  0.94731074  0.20158877]
  [ 0.15257418 -0.2438969   0.95772408]]

 [[ 0.41000757 -0.17353549  0.89542126]
  [ 0.67036564 -0.60836257 -0.42485867]
  [ 0.61846883  0.77445492 -0.13310103]]

 [[ 0.87029019 -0.34597063 -0.35056998]
  [ 0.41489509  0.89852303  0.14324257]
  [ 0.26543747 -0.27011237  0.92551729]]

 [[ 0.02913678  0.39925367  0.91637741]
  [ 0.97404457  0.19453631 -0.11572728]
  [-0.22447322  0.89596436 -0.3832227 ]]]

Positions

[[ 0.25325665 -0.46522858 -0.07112432]
 [-0.0887955  -0.7953031   0.58309975]
 [ 0.02976562 -0.3343

The following cell calculates the tool offset for the values reported by the CalibrateTooltip script in Unity.
You should paste the position values directly into the local transform of the tooltip.

In [15]:

#positions = ""
#rotations = ""

positions = "1.468334,0.8307886,-1.051155,1.515763,0.8493054,-1.090657,1.540523,0.8502798,-1.014596,1.478976,0.8552772,-1.037528"
rotations = "0.6241189,-0.1459335,0.76758,0.6069545,0.7091923,-0.3586816,-0.4920182,0.6897461,0.5311953,0.3399733,0.4658172,0.8169654,-0.270656,0.8804199,-0.3893664,-0.9006462,-0.08874249,0.4253954,0.7232004,0.47163,0.5045258,0.1435715,0.6118959,-0.7777985,-0.6755505,0.6349398,0.3748108,0.3829958,-0.1557047,0.9105329,0.1792883,0.9794778,0.09208079,-0.9061841,0.1279814,0.403052"

pm = np.fromstring(positions, sep=',')
Rm = np.fromstring(rotations, sep=',')

samples = int(pm.shape[0] / 3);

pm = pm.reshape(samples, 3)
Rm = Rm.reshape(samples, 3, 3)

PW, PT = tool_offset(Rm, pm)

print("\nTool offset to paste into Unity")
print(PT[0,0], ",", PT[0,1], ",", PT[0,2])




IR

[[ 1.          0.          0.         -0.6241189   0.1459335  -0.76758   ]
 [ 0.          1.          0.         -0.6069545  -0.7091923   0.3586816 ]
 [ 0.          0.          1.          0.4920182  -0.6897461  -0.5311953 ]
 [ 1.          0.          0.         -0.3399733  -0.4658172  -0.8169654 ]
 [ 0.          1.          0.          0.270656   -0.8804199   0.3893664 ]
 [ 0.          0.          1.          0.9006462   0.08874249 -0.4253954 ]
 [ 1.          0.          0.         -0.7232004  -0.47163    -0.5045258 ]
 [ 0.          1.          0.         -0.1435715  -0.6118959   0.7777985 ]
 [ 0.          0.          1.          0.6755505  -0.6349398  -0.3748108 ]
 [ 1.          0.          0.         -0.3829958   0.1557047  -0.9105329 ]
 [ 0.          1.          0.         -0.1792883  -0.9794778  -0.09208079]
 [ 0.          0.          1.          0.9061841  -0.1279814  -0.403052  ]]

IP

[[ 1.468334 ]
 [ 0.8307886]
 [-1.051155 ]
 [ 1.515763 ]
 [ 0.8493054]
 [-1.090657 ]
 [ 1.